## 基于sklearn的文本分类---支持向量机SVM(3)

>本文是文本分类的第三篇，记录使用朴素贝叶斯进行文本分类任务，数据集下载地址:http://thuctc.thunlp.org/

>文本分类的主要内容如下:
    - 1.基于逻辑回归的文本分类
    - 2.基于朴素贝叶斯的文本分类
    - 3.基于SVM的文本分类
    - 4.基于最大熵的文本分类
    - 5.使用LDA进行文档降维以及特征选择
    - 6.基于多层感知机MLPC的文本分类
    - 7.基于卷积神经网络词级别的文本分类以及调参
    - 8.基于卷积神经网络的句子级别的文本分类以及调参
    - 9.基于Facebook fastText的快速高效文本分类
    - 10.基于RNN的文本分类
    - 11.基于LSTM的文本分类
    - 12.总结

### 1 数据预处理
>其中使用的训练数据来自清华大学开源的文本分类数据集，具体的数据处理代码见上面的1-基于逻辑回归的文本分类。

In [1]:
import os
import codecs
import jieba
import re

from sklearn.utils import shuffle

In [2]:
def split_data_with_label(corpus):
    """
    将数据划分为训练数据和样本标签
    :param corpus: 
    :return: 
    """
    input_x = []
    input_y = []

    tag = []
    if os.path.isfile(corpus):
        with codecs.open(corpus, 'r') as f:
            for line in f:
                tag.append(line)
                
    else:
        for docs in corpus:
            for doc in docs:
                tag.append(doc)
    tag = shuffle(tag)
    for doc in tag:
        index = doc.find(' ')
        input_y.append(doc[:index])
        input_x.append(doc[index + 1 :])

    # 打乱数据，避免在采样的时候出现类别不均衡现象
    # datasets = np.column_stack([input_x, input_y])
    # np.random.shuffle(datasets)
    # input_x = []
    # input_y = []
    # for i in datasets:
    #     input_x.append(i[:-1])
    #     input_y.append(i[-1:])
    return [input_x, input_y]

>这个函数返回两个值，其中第一个返回值input_x是样本数据，一共14*1000行，第二个参数input_y和input_x有着相同的行数，每行对应着input_x中新闻样本的类别标签.

### 2.特征选择

>下面将进行特征提取，特征选择的方法有基本的bag-of-words, tf-idf,n-gran等，我们在使用朴素贝叶斯做相关特征选择的时候，进行了相关方法的比较，这里不再重复，特征选择采用TF-IDF,n-gram选择unigram和bigram的条件下进行SVM的文本分类实验，下面是代码:

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics.scorer import make_scorer
from sklearn.svm import SVC
from sklearn import metrics

from time import time

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
def feature_extractor(input_x, case='tfidf', n_features=1000):
    """
    特征抽取
    :param corpus: 
    :param case: 不同的特征抽取方法
    :return: 
    """
    # ngram_range=n_gram
    return TfidfVectorizer(max_features=n_features).fit_transform(input_x)

> 接下来将进行训练数据和测试数据的切分，现在不进行更好的交叉验证等技术，仅仅简单的以一定的比例划分训练数据和测试数据。使用sklearn中提供的工具，具体代码如下:

In [5]:
def split_data_to_train_and_test(corpus, indices=0.2, random_state=10, shuffle=True):
    """
    将数据划分为训练数据和测试数据
    :param corpus: [input_x]
    :param indices: 划分比例
    :random_state: 随机种子
    :param shuffle: 是否打乱数据
    :return: 
    """
    input_x, y = corpus

    # 切分数据集
    x_train, x_dev, y_train, y_dev = train_test_split(input_x, y, test_size=indices, random_state=10)
    print("Vocabulary Size: {:d}".format(input_x.shape[1]))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    return x_train, x_dev, y_train, y_dev

> 函数返回四个值，分别是训练数据的样本，训练数据的标签，测试数据样本，测试数据真实标签，下面调用朴素贝叶斯进行分类。  

> SVM是一种判别式模型，有硬间隔的SVM,软间隔的SVM,还有基于核方法的SVM用于处理非线性问题等，SVM基于支持向量，其分裂结果只和支持向量有关，其在数据中构造一个线性的超平面，来进行数据的区分，因为其能够在小数据的情况下也能训练到一个好的模型在神经网络没有火起来之前，是一个很受欢迎的模型。

> 在sklearn中，SVM有三种实现，分别是SVC,NuSVC,LinearSVM ,前者很相似，但是接受的参数和数学推导有区别，LinearSVM是另外一种实现，其是种线性核的SVM，不支持其他核函数。所以在线性分类中，三种方法时一样的，如果要用到诸如RBF核，那么只能选择前两者了。 

> 这里主要是进行相关的实验，不在理论上展开太多，下面采用SVC进行文档分类，具体代码如下:  


In [6]:
def fit_and_predicted(train_x, train_y, test_x, test_y, C=1.0, gamma='auto', decision_function_shape='ovr', kernel='linear'):
    """
    训练与预测
    :param train_x: 
    :param train_y: 
    :param test_x: 
    :param test_y: 
    :param C: 松弛变量C
    :param gamma: 核系数
    :param decision_function_shape: 多分类的决策，ovr（one-vs-rest）default  ovo（one-vs-one） 
    :return: 
    """
    clf = SVC(C=C, gamma=gamma, decision_function_shape=decision_function_shape, kernel=kernel).fit(train_x, train_y)
    predicted = clf.predict(test_x)
    print(metrics.classification_report(test_y, predicted))
    print('accuracy_score: %0.5fs' %(metrics.accuracy_score(test_y, predicted)))

> 上面函数调用SVC()，C,gamma,decision_function_shape三个参数，其中前两个是SVM调参的主要参数，其的值影响了SVM分类器的效果，我们将在后面进行具体的调参工作，最后一个参数是在进行多分类的时候用到的方法， 其有两个可选值:ovo和ovr，ovo表示的是one-vs-one，表示每个二分类只拿两个类别的数据来训练，N个类别训练下来一共要训练x次，ovr表示one-vs-rest,表示每次二分类训练器拿一个类别和其与类别一起训练二分类，N个类别一共需要训练N个分类器，但是前者每次训练的数据量较小，后者虽然训练次数少，但是每次训练的数据量较大，具体选择哪一个，需要根据需要训练的数据量和训练机器的内存等因素来决定

> 下面进行文本分类的具体流程，将进行实际的代码运行阶段了。


In [7]:
# 1. 加载语料
corpus = split_data_with_label('thu_data_2000')

In [8]:
# 4. 训练以及测试
t0 = time()
print('\t\tC=1.0,gama=\'auto\'的SVM文本分类\t\t')
input_x, y = corpus
# 2. 特征选择
input_x = feature_extractor(input_x, 'tfidf')
# 3.切分训练数据和测试数据
train_x, test_x, train_y, test_y = split_data_to_train_and_test([input_x, y])
fit_and_predicted(train_x, train_y, test_x, test_y)
print('time uesed: %0.4fs' %(time() - t0))

		C=1.0,gama='auto'的SVM文本分类		
Vocabulary Size: 1000
Train/Dev split: 22400/5600
             precision    recall  f1-score   support

__label__体育       0.93      0.94      0.94       422
__label__娱乐       0.86      0.88      0.87       377
__label__家居       0.79      0.87      0.83       391
__label__彩票       0.99      0.94      0.96       392
__label__房产       0.94      0.92      0.93       395
__label__教育       0.94      0.92      0.93       391
__label__时尚       0.85      0.84      0.85       395
__label__时政       0.82      0.86      0.84       418
__label__星座       0.96      0.95      0.95       391
__label__游戏       0.94      0.88      0.91       420
__label__社会       0.81      0.85      0.83       388
__label__科技       0.82      0.82      0.82       415
__label__股票       0.83      0.88      0.85       412
__label__财经       0.92      0.83      0.87       393

avg / total       0.89      0.88      0.88      5600

accuracy_score: 0.88357s
time uesed: 126.9834s


> 必须说明的是，和前几篇分类比起来，我在特征选择哪里选择了n_feature=1000,以此来降低维度，因为在高维情况下，SVM运行的非常的。。非常的慢。。。因为这个特征的维度已经上升到了500W。
> 而且使用RBF核的时候，不仅训练慢，效果也巨不好？

### 3 特征降维

> 从上面的实验可以看出，特征维度为500W维，这种维度的数据，称为维度灾难，应该不为过了，可以发现在训练的过程中非常的忙。自然而然的就应该想到现在要进行降维了。比较常见的降维方法有PCA降维，当然了，在文本中，降维方法很多很多，本文将会使用LDA方法，训练出文档主题矩阵作为文档的表示，下面将会选用后者。了解LDA请大家去参考轮问吧。下面使用两种方法，进行文档降维：

#### 3.1 PCA降维

> 必须说明的是，sklearn 的PCA是不支持稀疏矩阵的输入的，文档说需要使用TruncatedSVD,也就是使用奇异值分解

In [27]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation

In [28]:
def feature_select_usePCA(input_x, n_components=1000):
    tf_vect = TfidfVectorizer().fit_transform(input_x)
    #pca = PCA(n_components=n_components)
    svd = TruncatedSVD(n_components=n_components)
    return svd.fit_transform(tf_vect)

In [29]:
# 4. 训练以及测试
t0 = time()
print('\t\t使用SVD降维的SVM文本分类\t\t')
input_x, y = corpus
# 2. 特征选择以及降维
input_x = feature_select_usePCA(input_x)
# 3.切分训练数据和测试数据
train_x, test_x, train_y, test_y = split_data_to_train_and_test([input_x, y])
fit_and_predicted(train_x, train_y, test_x, test_y)
print('time uesed: %0.4fs' %(time() - t0))

		使用PCA降维的SVM文本分类		
Vocabulary Size: 1000
Train/Dev split: 22411/5603
             precision    recall  f1-score   support

       _体育_       0.95      0.96      0.95       412
       _娱乐_       0.85      0.93      0.89       393
       _家居_       0.86      0.94      0.90       399
       _彩票_       0.99      0.95      0.97       399
       _房产_       0.96      0.91      0.93       386
       _教育_       0.94      0.89      0.91       422
       _时尚_       0.94      0.91      0.93       387
       _时政_       0.83      0.91      0.87       372
       _星座_       0.95      0.96      0.96       414
       _游戏_       0.98      0.90      0.94       410
       _社会_       0.83      0.90      0.87       399
       _科技_       0.88      0.79      0.83       393
       _股票_       0.87      0.89      0.88       398
       _财经_       0.94      0.90      0.92       419

avg / total       0.91      0.91      0.91      5603

accuracy_score: 0.91058s
time uesed: 1169.0307s


> 可以看到维度下降到了1000维，比之前直接使用最大词频的1000分类性能有所提升,约提升了2%，但是在特征降维的时候时间消耗比较大

#### 3.2 LDA降维

> LDA的目标是求解出两个隐含变量：theta:文档主题矩阵，表示文档的主题分布，phi:主题词分布，表示单词在主题下的分布，通过sklearn 中的LatentDirichletAlloc，通过fit_transform()函数，返回[n_samples, new_features_x]，其new_features_x的维度是确定的主题个数

In [39]:
def feature_select_useLDA(input_x, n_components=100):
    tf_vect = TfidfVectorizer().fit_transform(input_x)
    #pca = PCA(n_components=n_compLatentDirichletAllocationents)
    lda = LatentDirichletAllocation(n_components=n_components, learning_method='batch', n_jobs=-1)
    return lda.fit_transform(tf_vect)

In [ ]:
# 4. 训练以及测试
t0 = time()
print('\t\t使用LDA降维的SVM文本分类\t\t')
input_x, y = corpus
# 2. 特征选择以及降维
input_x = feature_select_useLDA(input_x, 1000)
# 3.切分训练数据和测试数据
train_x, test_x, train_y, test_y = split_data_to_train_and_test([input_x, y])
fit_and_predicted(train_x, train_y, test_x, test_y)
print('time uesed: %0.4fs' %(time() - t0))

		使用LDA降维的SVM文本分类		


#### 3.3 使用LDA降维的LDA参数调优

> LDA需要传入的参数主要有主题个数，训练过程是使用bath还是online，狄里克莱先验，最大迭代次数，下面我们进行LDA参数调整

In [ ]:
def fit_LDA(corpus, param_grid, cv=5):
    input_x, y = corpus
    
    scoring = ['precision_macro', 'recall_macro', 'f1_macro']
    clf = LatentDirichletAllocation(max_iter=500, n_job=-1)
    grid = GridSearchCV(clf, param_grid, cv=cv, scoring='accuracy')
    
    tf_vect = TfidfVectorizer().fit_transform(input_x)
    scores = grid.fit(tf_vect, y)
    
    print('parameters:')
    best_parameters = grid.best_estimator_.get_params()
    for param_name in sorted(best_parameters):
        print('\t%s: %r' %(param_name, best_parameters[param_name]))
    return scores

In [ ]:
# 4. 训练以及测试
t0 = time()
print('\t\tLDA调参\t\t')

n_components = [14, 20, 30, 50, 100]
learning_method = ['online', 'batch']
learning_decay = [0.1, 0.2, 0.5, 1.0, 2.0]
param_grid = dict(n_components=n_components, learning_method=learning_method, learning_decay=learning_decay)

fit_LDA(corpus, param_grid, )
print('time uesed: %0.4fs' %(time() - t0))

> 通过两种降维方法可以看出，使用！！！

> 最后我们使用基于SVD的降维方法，采用线性核，进行SVM文本分类的交叉验证和调参工作

### 3. 使用交叉验证
> 上面的实验中，我们只是简单的选取20%的数据作为测试集和80%的数据作为训练集，这样做是存在偶然性结构的， 即可能划分数据集不能表示真实的数据分布，导致模型训练参数的泛化性不好，采用交叉验证可以避免数据集划分导致的问题，下面，就进行该实验，实验在上一步的基础上使用TF-IDF和unigram,bigram和trigram来进行特征选择。

In [89]:
def train_and_test_with_CV(corpus, cv=5, alpha=1, fit_prior=True):
    """
    
    """
    input_x, y = corpus
#     scoring = {'prec_macro': 'precision_macro',
#                'rec_micro': make_scorer(recall_score, average='macro')}
    scoring = ['precision_macro', 'recall_macro', 'f1_macro']
    clf = SVC(alpha=alpha, fit_prior=fit_prior)
    scores = cross_validate(clf, input_x, y, scoring=scoring,
                            cv=cv, return_train_score=True)
    sorted(scores.keys()) 
    return scores

In [47]:
input_x, y = corpus
# 2. 特征选择
input_x = feature_extractor(input_x, 'tfidf')
scores = train_and_test_with_CV([input_x, y])

In [48]:
scores

{'fit_time': array([ 0.69856882,  0.6891861 ,  0.68457079,  0.68122745,  0.68401599]),
 'score_time': array([ 0.24055672,  0.25055385,  0.24642444,  0.24583435,  0.25062966]),
 'test_f1_macro': array([ 0.93190598,  0.93358814,  0.92900074,  0.93620104,  0.93139325]),
 'test_precision_macro': array([ 0.93411186,  0.93509947,  0.93082131,  0.93790787,  0.93312355]),
 'test_recall_macro': array([ 0.93178571,  0.93357143,  0.92892857,  0.93607143,  0.93142857]),
 'train_f1_macro': array([ 0.95534592,  0.95516529,  0.95665886,  0.95573948,  0.95629695]),
 'train_precision_macro': array([ 0.95629235,  0.95618146,  0.95767379,  0.9566414 ,  0.95725075]),
 'train_recall_macro': array([ 0.95526786,  0.95508929,  0.95660714,  0.95571429,  0.95625   ])}

>交叉验证的K=10的时候

In [49]:
scores = train_and_test_with_CV([input_x, y],cv=10)

In [50]:
scores

{'fit_time': array([ 0.86708903,  0.85473442,  0.85248995,  0.8252821 ,  0.93414092,
         1.118325  ,  1.41779876,  1.2739253 ,  1.98447776,  1.11306906]),
 'score_time': array([ 0.16501474,  0.16674805,  0.17412877,  0.15616584,  0.14272356,
         0.21593046,  0.44325757,  0.30753231,  0.19881511,  0.20148587]),
 'test_f1_macro': array([ 0.93355446,  0.93725727,  0.9367952 ,  0.93744957,  0.9319552 ,
         0.93147271,  0.94146465,  0.93213457,  0.93504439,  0.93282066]),
 'test_precision_macro': array([ 0.93583195,  0.93947505,  0.93829325,  0.93885285,  0.9343669 ,
         0.93272889,  0.94303357,  0.93393932,  0.93704557,  0.93441742]),
 'test_recall_macro': array([ 0.93357143,  0.93714286,  0.93678571,  0.9375    ,  0.93178571,
         0.93142857,  0.94142857,  0.93214286,  0.935     ,  0.93285714]),
 'train_f1_macro': array([ 0.9565462 ,  0.95530877,  0.95550728,  0.95550059,  0.9569892 ,
         0.95626531,  0.95577014,  0.95573623,  0.95608533,  0.95600703]),
 'trai

### 4 寻找最好的参数

> 朴素贝叶斯的参数比偶较少，根据sklearn的文档可以看出，其参数主要是平滑项参数alpha、是否需要依靠样本去学习类别先验fit_prior和给定类别先验class_prio的给定.

> 下面对这些参数做相关实验。

In [84]:
from sklearn.grid_search import GridSearchCV
def train_and_predicted_with_graid(corpus, cv, param_grid):
    input_x, y = corpus
    
    scoring = ['precision_macro', 'recall_macro', 'f1_macro']
    clf = MultinomialNB()
    grid = GridSearchCV(clf, param_grid, cv=cv, scoring='accuracy')
    
    scores = grid.fit(input_x, y)
    
    print('parameters:')
    best_parameters = grid.best_estimator_.get_params()
    for param_name in sorted(best_parameters):
        print('\t%s: %r' %(param_name, best_parameters[param_name]))
    return scores

In [85]:
k_alpha = [0, 1,2,4,10]
fit_prior= [True, False]
param_grid = dict(alpha=k_alpha, fit_prior=fit_prior)
print(param_grid)
scores = train_and_predicted_with_graid([input_x, y], 5, param_grid)

{'fit_prior': [True, False], 'alpha': [0, 1, 2, 4, 10]}


/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.5/dist-packages/sklearn/nai

parameters:
	alpha: 0
	class_prior: None
	fit_prior: True


/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [88]:
print(scores)

{'test_recall_macro': array([ 0.93357143,  0.93714286,  0.93678571,  0.9375    ,  0.93178571,
        0.93142857,  0.94142857,  0.93214286,  0.935     ,  0.93285714]), 'test_precision_macro': array([ 0.93583195,  0.93947505,  0.93829325,  0.93885285,  0.9343669 ,
        0.93272889,  0.94303357,  0.93393932,  0.93704557,  0.93441742]), 'train_recall_macro': array([ 0.95650794,  0.9552381 ,  0.95543651,  0.95543651,  0.95694444,
        0.95619048,  0.95571429,  0.95571429,  0.95603175,  0.95595238]), 'fit_time': array([ 0.86708903,  0.85473442,  0.85248995,  0.8252821 ,  0.93414092,
        1.118325  ,  1.41779876,  1.2739253 ,  1.98447776,  1.11306906]), 'train_f1_macro': array([ 0.9565462 ,  0.95530877,  0.95550728,  0.95550059,  0.9569892 ,
        0.95626531,  0.95577014,  0.95573623,  0.95608533,  0.95600703]), 'test_f1_macro': array([ 0.93355446,  0.93725727,  0.9367952 ,  0.93744957,  0.9319552 ,
        0.93147271,  0.94146465,  0.93213457,  0.93504439,  0.93282066]), 'train_pr

> 使用最佳参数进行训练

In [92]:
scores = train_and_test_with_CV([input_x, y], cv=10, alpha=0)
print(scores)

/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.5/dist-packages/sklearn/nai

{'test_recall_macro': array([ 0.98714286,  0.98607143,  0.98642857,  0.98607143,  0.98178571,
        0.98464286,  0.98535714,  0.98214286,  0.97714286,  0.98392857]), 'test_precision_macro': array([ 0.98735865,  0.98622353,  0.98659011,  0.98618267,  0.98201582,
        0.9849389 ,  0.98558907,  0.9825292 ,  0.97748478,  0.9840529 ]), 'train_recall_macro': array([ 0.99809524,  0.99781746,  0.99785714,  0.99789683,  0.9975    ,
        0.9975    ,  0.9977381 ,  0.99801587,  0.99785714,  0.99781746]), 'fit_time': array([ 0.81418514,  0.80592179,  0.81234241,  0.79919314,  0.8071866 ,
        0.79807925,  0.79640722,  0.77489328,  0.80264211,  0.7880013 ]), 'train_f1_macro': array([ 0.9980958 ,  0.99781816,  0.99785777,  0.99789754,  0.99750104,
        0.99750157,  0.99773862,  0.99801643,  0.99785776,  0.99781855]), 'test_f1_macro': array([ 0.98716361,  0.98606902,  0.98643996,  0.98603623,  0.98180131,
        0.98463883,  0.98534904,  0.98216567,  0.9771135 ,  0.98393848]), 'train_pr

>可以看到，训练得到的结果相对于在没有进行最优参数调整的时候提高了约5%，效果是明显的。

### 5. 总结

>本文记录了使用sklearn，采用朴素贝叶斯进行文本分类任务，在使用简单的bag-of-word,tf-idf 作为参数选择，为了增加特征，保留句子中的部分语义信息，
，我们还进行了n-gram操作，在特征选择阶段，我们发现，使用tf-idf的特征表示方法比简单的词袋模型要好，添加了n-gram特征后，效果也有一定的提升；

> 在选取好了特征后，我们对数据集进行交叉验证，发现cv=10相对cv=5的时候有细微的提升，但是效果不明显，说明本数据集在cv=5的时候已经够用了，不需要再继续使用CV=10增加计算量；

> 最后，我们进行了最佳参数的寻找，由于naive bayes 分类器的参数较少，调参起来相对简单，在选用了最佳的参数后，我们得出了相对最优的结果，在测试集上P,R,F值几乎都达到了98%以上。  
但是分析我们的最佳参数，其中平滑项参数我们选取的是0，在模型中说明是不需要进行数据的平滑处理，但是经验而言，当数据变大，在开放的数据中，平滑项是必不可少的，此处的0，只是作为最有参数选寻找的个例，不应该用作一般性结论。